Do custom install of `sage-importance`

```bash
git clone https://github.com/karelze/sage.git
cd sage
pip install .
```

In [ ]:
import os
import sys
import pickle
from pathlib import Path

from catboost import CatBoostClassifier, Pool

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc

import pandas as pd
import torch
from torch import nn

sys.path.append("..")
from otc.models.classical_classifier import ClassicalClassifier

from sage import GroupedMarginalImputer, PermutationEstimator

from otc.features.build_features import (
    features_categorical,
    features_classical,
    features_classical_size,
    features_ml,
)

from otc.data.dataset import TabDataset
from otc.data.dataloader import TabDataLoader
from otc.features.build_features import features_classical_size

import wandb
from tqdm.auto import tqdm

In [ ]:
SEED = 42

np.random.seed(42) 

# set globally here
EXCHANGE = "ise"  
STRATEGY = "supervised"  
SUBSET = "test"  


# Change depending on model!
FEATURES = features_ml

In [ ]:
# set project name. Required to access files and artefacts
os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

## Sage Values🌵

In [ ]:
def get_feature_groups(feature_names, feature_str):

    fg_classical = {
        'chg_all_lead (grouped)': ['price_all_lead', 'chg_all_lead'],
        'chg_all_lag (grouped)': ['price_all_lag', 'chg_all_lag'],
        'chg_ex_lead (grouped)': ['price_ex_lead', 'chg_ex_lead'],
        'chg_ex_lag (grouped)': ['price_ex_lag', 'chg_ex_lag'],
        'quote_best (grouped)': ['BEST_ASK', 'BEST_BID', 'prox_best'],
        'quote_ex (grouped)': ['bid_ex', 'ask_ex','prox_ex' ],
        'TRADE_PRICE': ['TRADE_PRICE'],
        }
    
    fg_size = {'size_ex (grouped)': [ 'bid_ask_size_ratio_ex', 'rel_bid_size_ex',  'rel_ask_size_ex', 'bid_size_ex', 'ask_size_ex','depth_ex'], 'TRADE_SIZE': ['TRADE_SIZE']}
    
    fg_ml = {
        "STRK_PRC": ["STRK_PRC"],
        "ttm": ["ttm"],
        "option_type": ["option_type"],
        "root":["root"],
        "myn":["myn"],
        "day_vol":["day_vol"], 
        "issue_type":["issue_type"],
    }
    
    if feature_str.endswith("classical"):
        feature_groups = group_names = fg_classical    
    if feature_str.endswith("classical-size"):
        feature_groups = group_names = {**fg_classical , **fg_size}
    if feature_str.endswith("ml"):
        feature_groups = group_names = {**fg_classical, **fg_size, **fg_ml}      
    

    # Group indices
    groups = []
    for _, group in feature_groups.items():
        ind_list = []
        for feature in group:
            ind_list.append(feature_names.index(feature))
        groups.append(ind_list)

    return groups, group_names


In [ ]:
# load unscaled data for classical classifier
run = wandb.init(project="thesis", entity="fbv")

dataset = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_none:latest"

artifact = run.use_artifact(dataset)
data_dir = artifact.download()

data = pd.read_parquet(Path(data_dir, "test_set.parquet"), engine="fastparquet", columns=[*features_classical_size, "buy_sell"])

y_test = data["buy_sell"]
X_test = data.drop(columns="buy_sell")

feature_names = X_test.columns

### Classical Classifier🏦

In [ ]:
sample_size = 256

In [ ]:
idx = np.random.choice(y_test.index, size=sample_size, replace=False)

X_importance = X_test.loc[idx]
y_importance = y_test.loc[idx]

In [ ]:
# compare benchmarks
configs = [
    [("quote", "best"), ("quote", "ex"), ("rev_tick", "all")],
    [("trade_size", "ex"), ("quote", "best"),  ("quote", "ex"), ("depth", "best"), ("depth", "ex"), ("rev_tick", "all")]  
]

results = []
for config in configs:
    
    groups, group_names = get_feature_groups(X_importance.columns.tolist(), "classical-size")
    
    clf = ClassicalClassifier(layers=config, random_state=SEED, strategy="random")
    # only set headers etc, no leakage
    clf.fit(X=X_test.head(5), y=y_test.head(5))
    
    def call_classical(X):
        
        pred = clf.predict_proba(X)
        # max_class = np.argmax(pred, axis=-1)
        # return max_class
        return pred

    # apply group based imputation + estimate importances in terms of zero-one loss
    imputer = GroupedMarginalImputer(call_classical, X_importance.values, groups)
    estimator = PermutationEstimator(imputer, "zero one")
    
    # calculate values over entire test set
    sage_values = estimator(X_test.values, y_test.values.clip(0))
    
    # save sage values + std deviation to data frame
    result = pd.DataFrame(index=group_names, data={"values": sage_values.values, "std": sage_values.std})
    results.append(result)

In [ ]:
# generate names for df
names = []

# generate human readable names like quote(best)->quote(ex)
for r in tqdm(configs):
    name = "->".join("%s(%s)" % tup for tup in r)
    names.append(name)

results_df = pd.concat(results, axis=1, keys=names)

# flatten column names (required to save to parquet)
results_df.columns = [' '.join(col).strip() for col in results_df.columns.values]

In [ ]:
results_df

In [ ]:
KEY = f"{EXCHANGE}_{STRATEGY}_{SUBSET}_classical_feature_importance_{sample_size}"

URI_FI_CLASSICAL = f"gs://thesis-bucket-option-trade-classification/data/results/{KEY}.parquet"

results_df.to_parquet(URI_FI_CLASSICAL)

result_set = wandb.Artifact(name=KEY, type="results")
result_set.add_reference(URI_FI_CLASSICAL, name="results")

### Gradient Boosting 🐈

In [ ]:
FEATURE_MAP = {
    "classical": features_classical,
    "classical-size": features_classical_size,
    "ml": features_ml,
    "semi-classical": features_classical,
    "semi-classical-size": features_classical_size,
    "semi-ml": features_ml,
}

run = wandb.init(project="thesis", entity="fbv")

# load processed data for gradient-boosting
dataset = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_log_standardized_clipped:latest"

artifact = run.use_artifact(dataset)
data_dir = artifact.download()

data = pd.read_parquet(Path(data_dir, "test_set.parquet"), engine="fastparquet", columns=[*features_ml, "buy_sell"])

y_test = data["buy_sell"]
X_test = data.drop(columns="buy_sell")

feature_names = X_test.columns

In [ ]:
idx = np.random.choice(X_test.index, size=sample_size, replace=False)

X_importance = X_test.loc[idx]
y_importance = y_test.loc[idx]

In [ ]:
configs = [("classical", "1gzk7msy_CatBoostClassifier_default.cbm:latest"),
    ("classical-size", "3vntumoi_CatBoostClassifier_default.cbm:latest"),
    ("ml", "2t5zo50f_CatBoostClassifier_default.cbm:latest"),
    ("semi-classical", "37lymmzc_CatBoostClassifier_default.cbm:latest"),
    ("semi-classical-size", "1vmti6db_CatBoostClassifier_default.cbm:latest"),
    ("semi-ml", "t55nd8r0_CatBoostClassifier_default.cbm:latest")]

results = []

for feature_str, model in configs:
    
    # get feature names and slice to subset
    fs = FEATURE_MAP.get(feature_str)
    X_importance_fs = X_importance.loc[:, fs]
    X_importance_cols = X_importance_fs.columns.tolist()
    
    # calculate cat indices
    if feature_str.endswith("ml"):
        cat_features = [t[0] for t in features_categorical]
        cat_idx = [X_importance_cols.index(f) for f in cat_features]
    
    # get groups
    groups, group_names = get_feature_groups(X_importance_cols, feature_str)
    
    #  load model by identifier from wandb
    model_name = model.split("/")[-1].split(":")[0]
    
    artifact = run.use_artifact(model)
    model_dir = artifact.download()
    clf = CatBoostClassifier()
    clf.load_model(fname=Path(model_dir, model_name))
    
    
    # use callable instead of default catboost as it doesn't work with categoricals otherwise
    pred=None
    
    def call_catboost(X):
        if feature_str.endswith("ml"):       
            # convert categorical to int
            X = pd.DataFrame(X, columns=X_importance.columns)
            # Update the selected columns in the original DataFrame
            X[cat_features] = X.iloc[:, cat_idx].astype(int)
            # pass cat indices
            return clf.predict_proba(Pool(X, cat_features=cat_idx))
        else:
            return clf.predict_proba(X)
            
    
    # apply group based imputation + estimate importances in terms of zero-one loss
    imputer = GroupedMarginalImputer(call_catboost, X_importance_fs, groups)
    # imputer = MarginalImputer(call_catboost, X_importance_fs)
    estimator = PermutationEstimator(imputer, "zero one")
    
    # calculate values over entire test set
    sage_values = estimator(X_test.loc[:,fs].values, y_test.clip(0).values)
    
    # save sage values + std deviation to data frame
    result = pd.DataFrame(index=group_names, data={"values": sage_values.values, "std": sage_values.std})
    # result = pd.DataFrame(index=X_importance_cols, data={"values": sage_values.values, "std": sage_values.std})
    results.append(result)

In [ ]:
names = [f"gbm({feature_str[0]})" for feature_str in configs]
results_df = pd.concat(results, axis=1, keys=names)
results_df.columns = [' '.join(col).strip() for col in results_df.columns.values]

In [ ]:
results_df

In [ ]:
# list to data frame + set human readable names
names = [f"gbm({feature_str[0]})" for feature_str in configs]
results_df = pd.concat(results, axis=1, keys=names)
results_df.columns = [' '.join(col).strip() for col in results_df.columns.values]

# save to google clound and save identiifer
KEY = f"{EXCHANGE}_{STRATEGY}_{SUBSET}_gbm_feature_importance_{sample_size}"

URI_FI_GBM = f"gs://thesis-bucket-option-trade-classification/data/results/{KEY}.parquet"

results_df.to_parquet(URI_FI_GBM)

result_set = wandb.Artifact(name=KEY, type="results")
result_set.add_reference(URI_FI_GBM, name="results")

In [ ]:
results_df

### Transformer Classifier 🤖

In [ ]:
configs = [
    ("classical", "3jpe46s1_TransformerClassifier_default.pkl:latest"),
    ("classical-size", "1qx3ul4j_TransformerClassifier_default.pkl:latest"), 
    ("ml", "2h81aiow_TransformerClassifier_default.pkl:latest"),
    ("semi-classical", "12isqh2m_TransformerClassifier_default.pkl:latest"),
    ("semi-classical-size", "2hv1nayy_TransformerClassifier_default.pkl:latest"), 
    ("semi-ml", "3jbqpp4r_TransformerClassifier_default.pkl:latest"),
]

results = []

for feature_str, model in configs:
    # load model by identifier from wandb
    model_name = model.split("/")[-1].split(":")[0]

    # get feature names and slice to subset
    fs = FEATURE_MAP.get(feature_str)
    X_importance_fs = X_importance.loc[:, fs]
    X_importance_cols = X_importance_fs.columns.tolist()
    
    # calculate cat indices
    if feature_str.endswith("ml"):
        cat_features = [t[0] for t in features_categorical]
        cat_idx = [X_importance_cols.index(f) for f in cat_features]
    
    # get groups
    groups, group_names = get_feature_groups(X_importance_cols, feature_str)
    
    model_name = model.split("/")[-1].split(":")[0]

    artifact = run.use_artifact(model)
    model_dir = artifact.download()

    with open(Path(model_dir, model_name), 'rb') as f:
        clf = pickle.load(f)
    
    # apply group based imputation + estimate importances in terms of zero-one loss
    imputer = GroupedMarginalImputer(clf, X_importance_fs, groups)
    estimator = PermutationEstimator(imputer, "zero one")
    
    # calculate values over entire test set
    sage_values = estimator(X_test.loc[:,fs].values, y_test.clip(0).values)
    
    # save sage values + std deviation to data frame
    result = pd.DataFrame(index=group_names, data={"values": sage_values.values, "std": sage_values.std})
    results.append(result)

In [ ]:
# list to data frame + set human readable names
names = [f"fttransformer({feature_str[0]})" for feature_str in configs]
results_df = pd.concat(results, axis=1, keys=names)
results_df.columns = [' '.join(col).strip() for col in results_df.columns.values]

# save to google clound and save identiifer
KEY = f"{EXCHANGE}_{STRATEGY}_{SUBSET}_fttransformer_feature_importance_{sample_size}"

URI_FI_FTTRANSFORMER = f"gs://thesis-bucket-option-trade-classification/data/results/{KEY}.parquet"

results_df.to_parquet(URI_FI_FTTRANSFORMER)

result_set = wandb.Artifact(name=KEY, type="results")
result_set.add_reference(URI_FI_FTTRANSFORMER, name="results")
run.log_artifact(result_set)

wandb.finish()

In [ ]:
results_df

## Attention Maps for Transformers

We calculate the average attention map from all transformer blocks, as done in the [here](https://github.com/hila-chefer/Transformer-MM-Explainability/blob/main/lxmert/lxmert/src/ExplanationGenerator.py#L26) and [here](https://colab.research.google.com/github/hila-chefer/Transformer-MM-Explainability/blob/main/CLIP_explainability.ipynb#scrollTo=fWKGyu2YAeSV)

In [ ]:
params = {
    "pgf.texsystem": "xelatex",
    "pgf.rcfonts": False,
    "font.serif": [],
    "font.family": "serif",
    "font.sans-serif": [],
    "axes.labelsize": 11,
}

plt.rcParams.update(params)
rc("text", usetex=True)

plt.rc('text.latex', preamble=r'\usepackage{amsmath}\usepackage[utf8]{inputenc}')

CM = 1 / 2.54

cmap = mpl.colormaps.get_cmap("plasma")

In [ ]:
MODEL = "2h81aiow_TransformerClassifier_default.pkl:latest"

run = wandb.init(project="thesis", entity="fbv")

model_name = MODEL.split("/")[-1].split(":")[0]

artifact = run.use_artifact(MODEL)
model_dir = artifact.download()
    
with open(Path(model_dir, model_name), 'rb') as f:
    model = pickle.load(f)
    
clf = model.clf

In [ ]:
dataset = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_log_standardized:latest"

artifact = run.use_artifact(dataset)
data_dir = artifact.download()

data = pd.read_parquet(Path(data_dir, "test_set.parquet"), engine="fastparquet", columns=[*features_ml, "buy_sell"])

y_test = data["buy_sell"]
X_test = data.drop(columns="buy_sell")

In [ ]:
X_test.head()

In [ ]:
key = "ise_quotes_mid"

# at quotes
# idx = [39342191, 39342189, 39342188, 39342175, 39342174, 39342171,
#             39342233, 39342241, 39342238, 39342239, 39342237, 39342193,
#             39342194, 39342199, 39342202, 39342204, 39342205, 39342218,
#             39342216, 39342214, 39342211, 39342212, 39342263, 39342269,
#             39342273, 39342281, 39342285, 39342291, 39342305, 39342304,
#             39342359, 39342349, 39342388, 39342389, 39342406, 39342407,
#             39342475, 39342493, 39342507, 39342523, 39342541, 39342564,
#             39342572, 39342585, 39342584, 39342612, 39342614, 39342615,
#             39342617, 39342623, 39342624, 39342633, 39342642, 39342651,
#             39342650, 39342661, 39342701, 39342717, 39342724, 39342739,
#             39342755, 39342754, 39342756, 39342764]


# at mid
idx =  [39342276, 39342363, 39342387, 39342437, 39342436, 39342428,
            39342464, 39342540, 39342608, 39342598, 39342620, 39342632,
            39342674, 39342781, 39342804, 39342824, 39342818, 39342821,
            39342861, 39342871, 39342894, 39342898, 39342931, 39342934,
            39342948, 39342954, 39342960, 39342969, 39342986, 39342987,
            39342991, 39342992, 39343036, 39343082, 39343100, 39343098,
            39343099, 39343101, 39343102, 39343109, 39343112, 39343124,
            39343128, 39343165, 39343193, 39343199, 39343211, 39343215,
            39343234, 39343242, 39343298, 39343346, 39343370, 39343390,
            39343412, 39343413, 39343415, 39343414, 39343426, 39343433,
            39343465, 39343464, 39343485, 39343498]

In [ ]:
# idx = 0
device = "cuda"
batch_size = len(idx)

cat_features = model.module_params["cat_features"]
cat_unique_counts = model.module_params["cat_cardinalities"]

dl_params = {
    "batch_size": batch_size,  
    "shuffle": False,
    "device": device,
}

test_data = TabDataset(X_test[X_test.index.isin(idx)], y_test[y_test.index.isin(idx)], cat_features=cat_features, cat_unique_counts=cat_unique_counts)


test_loader = TabDataLoader(
    test_data.x_cat,
    test_data.x_cont,
    test_data.weight,
    test_data.y,
    **dl_params
)



In [ ]:
x_cat, x_cont, weight, y = next(iter(test_loader))

In [ ]:
criterion = nn.BCEWithLogitsLoss()

# calculate outputs
logits = clf(x_cat, x_cont).flatten()

# zero gradients
clf.zero_grad()

# loss + backward pass
loss = criterion(logits, y)
loss.backward()

In [ ]:
# https://github.com/hila-chefer/Transformer-MM-Explainability/blob/main/lxmert/lxmert/src/ExplanationGenerator.py#L26
# https://colab.research.google.com/github/hila-chefer/Transformer-MM-Explainability/blob/main/CLIP_explainability.ipynb#scrollTo=fWKGyu2YAeSV

attn_block = clf.transformer.blocks[0].attention.get_attn()
# cat + cont + [CLS]
n_tokens = attn_block.shape[-1]
# residual connection. Repeat along batch dimension
res = torch.eye(n_tokens, n_tokens).to(device)
res = res.unsqueeze(0).expand(batch_size, n_tokens, n_tokens)

# one_hot = expected_outputs.sum()
# one_hot.backward(retain_graph=True)

cams = []
grads = []

for i, block in enumerate(clf.transformer.blocks):

    grad = block.attention.get_attn_gradients().detach()
    cam = block.attention.get_attn().detach()
    
    cams.append(cam)
    grads.append(grad)
    
    # reshape to [batch_size x num_head, num_tokens, num_tokens]
    cam = cam.reshape(-1, cam.shape[-1], cam.shape[-1])
    grad = grad.reshape(-1, grad.shape[-1], grad.shape[-1])
    
    # dot product
    cam = grad * cam
    
    # reshape to [batch_size, num_head, num_tokens, num_tokens]
    cam = cam.reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
    # clamp negative values, calculate mean over heads
    cam = cam.clamp(min=0).mean(dim=1)
    res = res + torch.bmm(cam, res)

relevancy = res

In [ ]:
# get first attention map from batch and visualize
batch_probs = relevancy.detach().cpu().numpy()

In [ ]:
# visualize
stack = []
max_stack = 16

for i in range(max_stack):
    row = batch_probs[-i][0,1:]
    # row = test[np.newaxis,...]
    stack.append(row)
    
stack_np = np.vstack(stack)

In [ ]:
cont_features = [f for f in X_test.columns.tolist() if f not in cat_features]
# see feature tokenizer but without cls token
labels = [*cont_features, *cat_features]

In [ ]:
labels_sanitized = ['trade price',
 'bid (ex)',
 'ask (ex)',
 'ask (best)',
 'bid (best)',
 'price lag (ex)',
 'price lead (ex)',
 'price lag (all)',
 'price lead (all)',
 'chg lead (ex)',
 'chg lag (ex)',
 'chg lead (all)',
 'chg lag (all)',
 'prox (ex)',
 'prox (best)',
 'bid ask size ratio (ex)',
 'rel. bid size (ex)',
 'rel. ask size (ex)',
 'trade size',
 'bid size (ex)',
 'ask size (ex)',
 'depth (ex)',
 'strike price',
 'time to maturity',
 'moneyness',
 'day volume',
 'option type',
 'issue type',
 'root']

In [ ]:
stack_np_copy = stack_np.copy()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14*CM,10*CM), sharey=True)
ax[0].imshow(stack_np.T, cmap='Blues', interpolation='nearest')
ax[0].yaxis.set_ticks(list(range(len(labels_sanitized))))
ax[0].set_yticklabels(labels_sanitized)
ax[0].set_xlabel("At Quotes")
ax[1].imshow(stack_np_copy.T, cmap='Blues', interpolation='nearest')
ax[1].yaxis.set_ticks(list(range(len(labels_sanitized))))
ax[1].set_yticklabels(labels_sanitized, fontsize="x-small")
ax[1].set_xlabel("At Mid")
plt.tight_layout()
plt.savefig(f"../reports/Graphs/attention_maps_{key}.pdf", bbox_inches="tight")

In [ ]:
labels_detail = ["$\mathtt{[CLS]}$", *labels_sanitized]

In [ ]:
print(cams[0])

In [ ]:
cams[3].shape

In [ ]:
labels_left = ['$\\mathtt{[CLS]}$', *["..."]*(len(labels_detail) - 1)]

In [ ]:
labels_left

In [ ]:
labels_detail

In [ ]:
from matplotlib.pyplot import cm

plt.figure(figsize=(3*CM,10*CM))


yoffset = 0
# xoffset = ei * width * example_sep
xoffset = 0


# width = 1
# # example_sep = 3
# word_height = 1
# pad = 0.02


width = 1
example_sep = 3
word_height = 0.01
pad = 0.05

# by index
l = 3
h = 0

cam = cams[l].reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
attention = cam[0,h,:,:]
attention /= attention.sum(axis=-1, keepdims=True)


# print(attention)
color = iter(cm.rainbow(np.linspace(0, 1, heads * layer)))

for position, word in enumerate(labels_left):
    plt.text(0, yoffset - position * word_height, word,
                ha="right", va="center", size="x-small")
for position, word in enumerate(labels_detail):
    plt.text(width, yoffset - position * word_height, word,
                ha="left", va="center", size="x-small")
# focus on cls token
c = next(color)
# CLS is prepended, get first row, similar to chefer
for i, vec in enumerate(attention[0:1]):
    for j, el in enumerate(vec):
        plt.plot([xoffset + pad, xoffset + width - pad],
                    [yoffset - word_height * i, yoffset - word_height * j],
                    color=c, linewidth=2, alpha=el.item())
plt.axis('off')
plt.tight_layout()
plt.savefig(f"../reports/Graphs/attention_head_{h+1}_layer_{l+1}_{key}.pdf", bbox_inches="tight")

In [ ]:
from matplotlib.pyplot import cm

plt.figure(figsize=(36,6))


yoffset = 0
# xoffset = ei * width * example_sep
xoffset = 0


# width = 1
# # example_sep = 3
# word_height = 1
# pad = 0.02


width = 3
example_sep = 3
word_height = 1
pad = 0.1

layer = 4
heads = 8

fig, axes = plt.subplots(layer, heads)


color = iter(cm.rainbow(np.linspace(0, 1, heads * layer)))

for l in range(layer):

    for h in range (heads):
        # [batch x head x attn x dim attn]

        cam = cams[l].reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])

        # [first in batch, head h, :,:]
        attention = cam[0,h,:,:]

        attention /= attention.sum(axis=-1, keepdims=True)

        # yoffset = 1
        # xoffset = h * width * example_sep

        # for position, word in enumerate(labels_detail):
        #     plt.text(xoffset + 0, yoffset - position * word_height, word,
        #             ha="right", va="center")
        #     plt.text(xoffset + width, yoffset - position * word_height, word,
        #             ha="left", va="center")

        # focus on cls token
        c = next(color)
        for i, vec in enumerate(attention[0:1]):
            for j, el in enumerate(vec):
                axes[l,h].plot([pad, width - pad], # x axis
                         [word_height * i, word_height * j],
                         color=c, linewidth=2, alpha=el.item())

        axes[l,h].set_title(f"head {l+1,h+1}", size='xx-small')
# fig.tight_layout()
        axes[l,h].set_xticks([])
        axes[l,h].set_yticks([])
        # axes[l,h].axis('off')

plt.savefig(f"../reports/Graphs/attention_heads_layer_all_{key}.pdf", bbox_inches="tight")

In [ ]:

data = {"grads":grads, "cams":cams, "final-scores":stack_np_copy}

In [ ]:
# Specify the file path where you want to save the pickle file
file_path = 'data.pickle'

# Open the file in binary mode and write the dictionary to it
with open(file_path, 'wb') as file:
    pickle.dump(data, file)